# tool_merge_encoded_yearly_messages_all_senders

# Main

In [1]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts

In [2]:
version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
share_directory =  '/home/jovyan/work/share/'
#share_directory =  '/home/jovyan/share/'

In [3]:
import pandas as pd
from pyspark.sql import functions

## merge all senders, fix types and save dataframe: v00001_v00001

In [ ]:
sparkSession = dfBasics.getSparkSession()
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00001/v00000/encoded/senders/enc/*.parquet').dropDuplicates()
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_0.parquet')                               

In [ ]:
_dtypes = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_0.parquet').dtypes
_dtypes = {c[0]:c[1] for c in _dtypes}

In [ ]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/1.1.24/*/*/*').dropDuplicates()

In [ ]:
for c in df.columns:
    df=df.withColumn(c, df[c].cast(_dtypes[c]))

In [ ]:
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_1.parquet')  

In [ ]:
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_*').dropDuplicates()
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00001.parquet')

## load dataframe : v00001_v00001

In [4]:
sparkSession = dfBasics.getSparkSession()
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00001.parquet')

## encoded values

## init encoders (run only once)

In [5]:
# - run this paragraph only once !
# - if you get an 'allow_pickle' error you need Kernel/restart kernel
import numpy as np
import encoder

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

### encode value

In [6]:
import numpy as np
def transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    return int( _encoder.transform([value])[0])

### decode value

In [7]:
def inverse_transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    if type(value) == int:
        return str(_encoder.inverse_transform(value))  
    elif type(value) == list:
        return [str(_encoder.inverse_transform(v)) for v in value]
    else:
        return None

In [8]:
def check_transformed(value0,column,value):
    return value0 == transform(value,column=column)

def calc_error(columns):
    status = columns[0]
    service = columns[1]
    error=int(not( ( check_transformed(status,'CSTATUS','PENDING') & check_transformed(service,'CSERVICE','InvoicePortal')  ) | \
        (check_transformed(status,'CSTATUS','PENDING') & check_transformed(service,'CSERVICE','IDS')  ) | \
        check_transformed(status,'CSTATUS','SUCCESS') | \
        check_transformed(status,'CSTATUS','SUCCESS_DOWNLOADED') | check_transformed(status,'CSTATUS','SUCCESS_POLLQUEUE')  ))
    return error

In [9]:
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
udf_add_error = udf(lambda y,z: calc_error((y,z)), IntegerType())

In [10]:
df3 = df.withColumn("error", udf_add_error(f.col("CSTATUS"), f.col("CSERVICE")).cast(IntegerType()))

## test dataframe

In [11]:
df3.where(f.col("year").isin([2020])).where(f.col("month").isin([1])).show(5)

+--------------------+-------------+-------------+-------+--------+-----------------+---------------+------------+-----------------+-------------------+-------+------------+----------------+----+-----+---+----+------+-----+
|    CGLOBALMESSAGEID|   CSTARTTIME|     CENDTIME|CSTATUS|CSERVICE|CSENDERENDPOINTID|CSENDERPROTOCOL|CINBOUNDSIZE|CRECEIVERPROTOCOL|CRECEIVERENDPOINTID|CSLATAT|CMESSAGETAT2|CSLADELIVERYTIME|year|month|day|hour|minute|error|
+--------------------+-------------+-------------+-------+--------+-----------------+---------------+------------+-----------------+-------------------+-------+------------+----------------+----+-----+---+----+------+-----+
|43ab3478-2c54-11e...|1577855101807|1577855110236|     14|       6|             3517|              0|         192|                0|                710|   8224|        8429|   1577855110031|2020|    1|  1|   6|     5|    0|
|4db209d3-2c54-11e...|1577855118372|1577855124039|     14|       5|             3427|              0|   

## store updated dataframe including errors

In [13]:
df4 = df3.where(f.col("year").isin([2020])).where(f.col("month").isin([1]))
df4.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00002_2020_1.parquet')

In [ ]:
#df3.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00002.parquet')

# obsolete

In [ ]:
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000.parquet')
#senders = df.select('CSENDERENDPOINTID').dropDuplicates().toPandas()['CSENDERENDPOINTID']
#len(senders)
df.show(n=20)

In [ ]:
senders = df.toPandas()['CSENDERENDPOINTID']
len(senders)

In [ ]:
sparkSession = dfBasics.getSparkSession()
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00001/v00000/encoded/senders/enc/*.parquet').dropDuplicates()
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_0.parquet')      

In [ ]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/1.1.24/*/*/*').dropDuplicates()
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_1.parquet')  

In [ ]:
senders = df.select('CSENDERENDPOINTID').dropDuplicates().toPandas()['CSENDERENDPOINTID']
len(senders)

In [ ]:
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_1.parquet')
#df.show(n=20)
df.dtypes

In [ ]:
_dtypes = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_0.parquet').dtypes
_dtypes = {c[0]:c[1] for c in _dtypes}

In [ ]:
df.dtypes

In [ ]:
_dtypes['CSENDERENDPOINTID']

In [ ]:
for c in df.columns:
    df=df.withColumn(c, df[c].cast(_dtypes[c]))

In [ ]:
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_1b.parquet')  

In [ ]:
!du -h  /home/jovyan/work/output/sla_enc_all_v00001_v00000_1.parquet

In [ ]:
sparkSession = dfBasics.getSparkSession()
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000_*').dropDuplicates()
df.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00000.parquet')

In [ ]:
!mv /home/jovyan/work/output/sla_enc_all_v00001_v00000.parquet /home/jovyan/work/output/sla_enc_all_v00001_v00001.parquet

In [ ]:
df = sparkSession.read.parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00001.parquet')
senders = df.select('CSENDERENDPOINTID').dropDuplicates().toPandas()['CSENDERENDPOINTID']
len(senders)
df.show(n=20)
#df.dtypes

In [ ]:
len (senders)

In [ ]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/*/*').dropDuplicates()
senders = df.toPandas()['CSENDERENDPOINTID']
senders = list(senders)

In [ ]:
len(senders)

In [ ]:
#senders

In [ ]:
import pyspark.sql.functions as f

def process(sender=None, dataframe=None):
    df3 = dataframe.where(f.col("CSENDERENDPOINTID").isin([sender]))
    return df3

df = process(sender=senders[0],dataframe=sparkSession.read.parquet('hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/*/*'))

In [ ]:
_result=df.toPandas()

In [ ]:
_result.head()

In [ ]:
failed = []
for sender in senders:
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS')
    except Exception as exception: 
        failed.append(sender)
        #if not 'temporary' in filename:
            #failed.append(filename.split('sla_enc_')[1].split('.')[0])

In [ ]:
#senders = sparkSession.read.parquet("/tmp/senders.parquet")
#senders = pd.read_parquet('/tmp/senders' + '.parquet', engine='pyarrow')
#senders = list(senders.toPandas()['CSENDERENDPOINTID'])

In [ ]:
senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').toPandas()['CSENDERENDPOINTID'])

In [ ]:
len(senders)

In [ ]:
#senders

In [ ]:
senders_files = sparkSession.read.options(delimiter=',') \
                      .csv('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/senders/senders_files.txt').toPandas()
senders_files.columns = ['size','filename']    

In [ ]:
#senders_files['filename'][0]

In [ ]:
#'sla_enc_' + senders[0] + '.parquet'

In [ ]:
failed = []
for sender in senders:
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS')
    except Exception as exception: 
        failed.append(sender)
        #if not 'temporary' in filename:
            #failed.append(filename.split('sla_enc_')[1].split('.')[0])

In [ ]:
failed.remove(None)
len(failed)

In [ ]:
working = [x for x in senders if x not in failed]
working.remove(None)

In [ ]:
len(working)

In [ ]:
#sender = working[0]
#filename = 'sla_enc_' + sender + '.parquet'
#_df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename ).toPandas()
#_df.columns

In [ ]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME', 'year', 'month', 'day', 'hour', 'minute']

In [ ]:
#columns

In [ ]:
df = pd.DataFrame(columns=columns)

In [ ]:
df = pd.read_parquet('/home/jovyan/work/output/sla_enc_all_5.parquet')

In [ ]:
processed_senders = pd.unique(df['CSENDERENDPOINTID'])

In [ ]:
len(processed_senders)

In [ ]:
len(pd.unique(df['CSENDERENDPOINTID']))

In [ ]:
# - run this paragraph only once !
# - if you get an 'allow_pickle' error you need Kernel/restart kernel

import encoder
import numpy as np

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

In [ ]:
import numpy as np
version_sla = 'v00001'
version     = version_sla + '/v00000'
share_directory =  '/home/jovyan/work/share/'

def transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    return int( _encoder.transform([value])[0])

#transform('e95a3090-ed87-11e8-a40c-5396ac1b495c',column='CSENDERENDPOINTID')

In [ ]:
for sender in working:
    try:
        if transform(sender,column='CSENDERENDPOINTID') not in processed_senders:
            filename = 'sla_enc_' + sender + '.parquet'
            _df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename ).toPandas()
            df = df.append(_df)
    except Exception as exception:
        exception
        print(sender,exception)       

In [ ]:
for sender in working:
    #print(sender)
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        _df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename ).toPandas()
        df = df.append(_df)
    except Exception as exception:
        pass

In [ ]:
#!ls -l /home/jovyan/work/output/sla_enc_all.parquet

In [ ]:
df.to_parquet('/home/jovyan/work/output/sla_enc_all_5.parquet')

In [ ]:
#pd.read_parquet('/home/jovyan/work/output/sla_enc_all_2.parquet')

In [ ]:
_senders = pd.unique(df['CSENDERENDPOINTID'])
len(_senders)

In [ ]:
#pd.unique(df[df['CSENDERENDPOINTID'] == _senders[0]]['CRECEIVERPROTOCOL'])

In [ ]:
#pd.unique(df['CRECEIVERPROTOCOL'])